In [1]:
"""
Link daily Fire Radiative Power (FRP) retrievals to daily fire perimeters
Author: maxwell.cook@colorado.edu
"""

import os, time, glob, gc
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pytz

from datetime import datetime
from shapely.geometry import box
from shapely.geometry import Polygon, MultiPolygon

import warnings
warnings.filterwarnings("ignore") # suppresses annoying geopandas warning

proj = 'EPSG:5070'

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/Aim2/')

print("Ready to go !")

Ready to go !


In [2]:
def create_bounds(gdf, buffer=None, by_bounds=True):
    """ Calculate a bounding rectangle for a given geometry and buffer """
    if by_bounds is True:
        geom = gdf.geometry.apply(lambda geo: box(*geo.bounds))
        if buffer is not None:
            geom = geom.buffer(buffer)
        # Apply the new geometry
        gdf_ = gdf.copy()
        gdf_.geometry = geom.geometry.apply(
            lambda geo: Polygon(geo) if geo.geom_type == 'Polygon' else MultiPolygon([geo]))
        return gdf_
    elif by_bounds is False:
        if buffer is not None:
            geom = gdf.geometry.buffer(buffer)
        else:
            gdf_ = gdf.copy()
            gdf_.geometry = gdf_.geometry.buffer(buffer)
            return gdf_    

 
def convert_datetime(acq_date, acq_time):
    """ Function to convert ACQ_DATE and ACQ_TIME to a datetime object in UTC """
    # Ensure ACQ_TIME is in HHMM format
    acq_time = str(acq_time) # force to string
    if len(acq_time) == 3:
        acq_time = '0' + acq_time
    elif len(acq_time) == 2:
        acq_time = '00' + acq_time
    elif len(acq_time) == 1:
        acq_time = '000' + acq_time

    acq_date_str = acq_date.strftime('%Y-%m-%d')
    dt = datetime.strptime(acq_date_str + acq_time, '%Y-%m-%d%H%M')
    dt_utc = pytz.utc.localize(dt)  # Localize the datetime object to UTC
    return dt_utc


print("Functions read !")

Functions read !


In [3]:
# Load the fire data.

In [4]:
aspen_events_fp = os.path.join(projdir,'data/spatial/mod/FIRED/fired-events_west_2018_to_2024_aspen1pct.gpkg')
aspen_events = gpd.read_file(aspen_events_fp)
aspen_events.columns

Index(['id', 'ig_date', 'ig_day', 'ig_month', 'ig_year', 'last_date',
       'event_dur', 'tot_pix', 'tot_ar_km2', 'fsr_px_dy', 'fsr_km2_dy',
       'mx_grw_px', 'mn_grw_px', 'mu_grw_px', 'mx_grw_km2', 'mn_grw_km2',
       'mu_grw_km2', 'mx_grw_dte', 'x', 'y', 'ig_utm_x', 'ig_utm_y', 'lc_code',
       'lc_mode', 'lc_name', 'lc_desc', 'lc_type', 'eco_mode', 'eco_name',
       'eco_type', 'tot_perim', 'na_l3name', 'pct_aspen', 'geometry'],
      dtype='object')

In [5]:
# Create a fire perimeter buffer to extract FRP observations.

In [6]:
bounds = create_bounds(aspen_events, by_bounds=True, buffer=1000)
bounds = bounds.set_crs(proj, allow_override=True) # ensure correct crs
bounds = bounds[['id','ig_date','last_date','geometry']] 
bounds.head()

,id,ig_date,last_date,geometry
0,282,2021-08-05,2021-08-05,"POLYGON ((-2086715.084 2356782.795, -2087666.8..."
1,492,2018-08-09,2018-08-13,"POLYGON ((-2099220.550 2280974.827, -2105343.3..."
2,1796,2021-10-28,2021-11-04,"POLYGON ((-1796487.029 1997450.102, -1799267.9..."
3,2207,2021-10-30,2021-10-30,"POLYGON ((-1822312.155 1967211.915, -1823260.8..."
4,2314,2021-10-29,2021-10-29,"POLYGON ((-1758198.372 1945155.099, -1759124.2..."


In [7]:
# Load the Active Fire Detection (AFD) data

AFDs from MODIS Collection 6.1 (1km) and the Suomi National Polar-Orbiting Partnership (VIIRS S-NPP 375m) were obtained from the NASA FIRMS (https://firms.modaps.eosdis.nasa.gov/download/) between 2018-2023 in the western US. The S-NPP AFDs include an archive shapefile and a NRT shapefile for Sept 2022-->

In [8]:
# Gather the archive and NRT S-NPP AFDs.
snpp_fp = os.path.join(projdir,'data/spatial/raw/NASA-FIRMS/DL_FIRE_SV-C2_476784/')
vects = glob.glob(snpp_fp+"*.shp")
print([os.path.basename(v) for v in vects])

['fire_nrt_SV-C2_476784.shp', 'fire_archive_SV-C2_476784.shp']


In [9]:
# Merge the NRT and archive vintages
archive = gpd.read_file([v for v in vects if "archive" in v][0]).to_crs(proj)
nrt = gpd.read_file([v for v in vects if "nrt" in v][0]).to_crs(proj)
snpp = pd.concat([archive, nrt], ignore_index=True)
snpp.head()

,LATITUDE,LONGITUDE,BRIGHTNESS,SCAN,TRACK,ACQ_DATE,ACQ_TIME,SATELLITE,INSTRUMENT,CONFIDENCE,VERSION,BRIGHT_T31,FRP,DAYNIGHT,TYPE,geometry
0,36.622398,-119.958420,319.25,0.45,0.63,2018-01-01,0854,N,VIIRS,n,1,276.46,1.93,N,0.0,POINT (-2100437.814 1774627.392)
1,48.166241,-102.698212,323.92,0.38,0.36,2018-01-01,0854,N,VIIRS,n,1,244.01,3.01,N,0.0,POINT (-502299.189 2814197.682)
2,48.152390,-102.697792,325.34,0.38,0.36,2018-01-01,0854,N,VIIRS,n,1,247.16,2.07,N,0.0,POINT (-502375.241 2812672.753)
3,47.879730,-102.706543,320.74,0.38,0.36,2018-01-01,0854,N,VIIRS,n,1,245.34,3.00,N,0.0,POINT (-505150.481 2782728.231)
4,44.249660,-104.516678,327.49,0.38,0.36,2018-01-01,0854,N,VIIRS,n,1,251.79,5.84,N,0.0,POINT (-677212.174 2392519.636)


In [11]:
# Set up a dictionary to store both MODIS and VIIRS AFDs
modis_fp = os.path.join(projdir,'data/spatial/raw/NASA-FIRMS/DL_FIRE_M-C61_476781/')

# Store these in a dictionary
afds = {
    "MOD61": gpd.read_file(modis_fp).to_crs(proj),
    "SNPP": snpp
}

afds['MOD61'].head()

,LATITUDE,LONGITUDE,BRIGHTNESS,SCAN,TRACK,ACQ_DATE,ACQ_TIME,SATELLITE,INSTRUMENT,CONFIDENCE,VERSION,BRIGHT_T31,FRP,DAYNIGHT,TYPE,geometry
0,44.3762,-119.1190,312.5,1.2,1.1,2018-01-01,0608,Terra,MODIS,85,6.03,269.1,21.9,N,0,POINT (-1819292.701 2598646.798)
1,32.1679,-107.7655,322.3,1.1,1.0,2018-01-01,1809,Terra,MODIS,77,6.03,289.1,19.0,D,0,POINT (-1101321.656 1078752.715)
2,32.1643,-107.7433,318.4,1.1,1.0,2018-01-01,1809,Terra,MODIS,74,6.03,287.9,14.9,D,0,POINT (-1099303.652 1078097.526)
3,31.7188,-102.0085,303.8,1.4,1.2,2018-01-01,1944,Aqua,MODIS,57,6.03,279.4,12.6,D,0,POINT (-566656.554 978378.293)
4,38.8238,-122.6829,302.0,2.7,1.6,2018-01-01,1946,Terra,MODIS,50,6.03,280.2,26.9,D,0,POINT (-2264952.497 2075236.763)


In [ ]:
# Extract the AFDs within fire bounds.

In [12]:
afd_events = {} # to store the filtered data
# Loop through sensors (MODIS and VIIRS)
for instrument, afd in afds.items():
    print(f"Processing: {instrument}")
    afd['VID'] = afd.index # add a unique ID
    
    # Remove low confidence observations
    try:
        afd = afd[afd['CONFIDENCE'] != 'l']
    except KeyError as e:
        print(f"KeyError: {e}")

    # Extract within fire bounds
    afd_ = gpd.sjoin(afd, bounds, how='inner', predicate='within')
    afd_.drop(columns=['index_right'], inplace=True)
    
    # Tidy the date columns for matching
    afd_['ACQ_DATE'] = pd.to_datetime(afd_['ACQ_DATE'])
    afd_['ACQ_MONTH'] = afd_['ACQ_DATE'].dt.month.astype(int)
    afd_['ACQ_YEAR'] = afd_['ACQ_DATE'].dt.year.astype(int)
    
    # Create a datetime object
    afd_.loc[:, 'ACQ_DATETIME'] = afd_.apply(lambda row: convert_datetime(row['ACQ_DATE'], row['ACQ_TIME']), axis=1)

    # Filter for matching dates
    afd_f = afd_[
        (afd_['ACQ_YEAR'] >= afd_['ig_date'].dt.year.astype(int)) & 
        (afd_['ACQ_MONTH'] >= afd_['ig_date'].dt.month.astype(int)) &
        (afd_['ACQ_YEAR'] <= afd_['last_date'].dt.year.astype(int)) &
        (afd_['ACQ_MONTH'] <= afd_['last_date'].dt.month.astype(int))
    ]

    # Drop any duplicates
    afd_f = afd_f.drop_duplicates(subset='VID', keep='first')
    print(f"\tThere are [{len(afd_f)}] high or nominal confidence detections within fire bounds.")

    # Overwrite the dictionary items
    afd_f['instrument'] = instrument # track which sensor the data is from
    afd_events[instrument] = afd_f

    del afd, afd_, afd_f
    gc.collect()

print("\nProcessing complete !")

Processing: MOD61
	There are [18594] high or nominal confidence detections within fire bounds.
Processing: SNPP
	There are [74806] high or nominal confidence detections within fire bounds.

Processing complete !


In [13]:
afd_events['MOD61'].columns

Index(['LATITUDE', 'LONGITUDE', 'BRIGHTNESS', 'SCAN', 'TRACK', 'ACQ_DATE',
       'ACQ_TIME', 'SATELLITE', 'INSTRUMENT', 'CONFIDENCE', 'VERSION',
       'BRIGHT_T31', 'FRP', 'DAYNIGHT', 'TYPE', 'geometry', 'VID', 'id',
       'ig_date', 'last_date', 'ACQ_MONTH', 'ACQ_YEAR', 'ACQ_DATETIME',
       'instrument'],
      dtype='object')

In [ ]:
# Calculate the daily FRP statistics

In [19]:
datetime_frp = {} # to store aggregated FRP by datetime
dob_frp = {} # by day of burn
for sensor, afd in afd_events.items():
    # Group by fire ID and calculate datetime FRP
    dt_frp = afd.groupby(['id', 'DAYNIGHT', 'ACQ_DATETIME'])['FRP'].quantile(0.99).reset_index(name='FRP_99_DT')
    
    # Add a date object
    dt_frp['DATE'] = dt_frp['ACQ_DATETIME'].dt.date

    def calc_frp_percentiles(group):
        return pd.Series({
            'FRP_MN': group['FRP_99_DT'].mean(),
            'FRP_SD': group['FRP_99_DT'].std(),
            'FRP_90': np.percentile(group['FRP_99_DT'], 90),
            'FRP_95': np.percentile(group['FRP_99_DT'], 95),
            'FRP_99': np.percentile(group['FRP_99_DT'], 99)
        })

    # Calculate the daily FRP statistics by day/night
    date_frp = dt_frp.groupby(['id', 'DATE', 'DAYNIGHT']).apply(calc_frp_percentiles).reset_index()
    
    # append to the dictionary
    datetime_frp[sensor] = dt_frp 
    dob_frp[sensor] = date_frp 
    
datetime_frp['MOD61'].head()

,id,DAYNIGHT,ACQ_DATETIME,FRP_99_DT,DATE
0,492,N,2018-08-10 05:38:00+00:00,69.649,2018-08-10
1,492,N,2018-08-10 09:50:00+00:00,30.448,2018-08-10
2,3518,N,2021-08-30 05:56:00+00:00,331.606,2021-08-30
3,3518,N,2021-08-30 10:14:00+00:00,28.700,2021-08-30
4,4131,D,2023-10-21 20:51:00+00:00,215.028,2023-10-21


In [20]:
dob_frp['MOD61'].head()

,id,DATE,DAYNIGHT,FRP_MN,FRP_SD,FRP_90,FRP_95,FRP_99
0,492,2018-08-10,N,50.0485,27.719293,65.7289,67.68895,69.25699
1,3518,2021-08-30,N,180.1530,214.186887,301.3154,316.46070,328.57694
2,4131,2023-10-21,D,215.0280,NaN,215.0280,215.02800,215.02800
3,4225,2018-08-07,D,197.5780,NaN,197.5780,197.57800,197.57800
4,4225,2018-08-07,N,7.3000,NaN,7.3000,7.30000,7.30000


In [21]:
dob_frp['MOD61'][dob_frp['MOD61']['id'] == 69951].head()

,id,DATE,DAYNIGHT,FRP_MN,FRP_SD,FRP_90,FRP_95,FRP_99
1324,69951,2018-08-19,N,7.494,NaN,7.4940,7.4940,7.49400
1325,69951,2018-08-21,N,431.795,NaN,431.7950,431.7950,431.79500
1326,69951,2018-08-22,N,10.600,NaN,10.6000,10.6000,10.60000
1327,69951,2018-08-23,D,131.468,54.925226,162.5384,166.4222,169.52924
1328,69951,2018-08-23,N,134.294,116.330379,200.1004,208.3262,214.90684


In [ ]:
# Merge to the daily fire perimeter data

In [22]:
daily_evt_fp = os.path.join(projdir, 'data/tabular/mod/EVT/fired-daily_west_2018_to_2023-EVT.csv')
daily_evt = pd.read_csv(daily_evt_fp)
daily_evt.head()

,Unnamed: 0,id,did,evt,count,total_pixels,pct_cover,EVT_NAME,EVT_PHYS,EVT_GP_N,EVT_CLASS
0,0,62372,0003d89c00208d55887fec4a95f58ac8,7017,2,5026,0.039793,Columbia Plateau Western Juniper Woodland and ...,Conifer,Juniper Woodland and Savanna,Open tree canopy
1,1,62372,0003d89c00208d55887fec4a95f58ac8,7080,4,5026,0.079586,Inter-Mountain Basins Big Sagebrush Shrubland,Shrubland,Big Sagebrush Shrubland and Steppe,Shrubland
2,2,62372,0003d89c00208d55887fec4a95f58ac8,7123,5,5026,0.099483,Columbia Plateau Steppe and Grassland,Grassland,Grassland and Steppe,Herbaceous-shrub-steppe
3,3,62372,0003d89c00208d55887fec4a95f58ac8,7127,5,5026,0.099483,Inter-Mountain Basins Semi-Desert Shrub-Steppe,Shrubland,Desert Scrub,Shrubland
4,4,62372,0003d89c00208d55887fec4a95f58ac8,7296,36,5026,0.716275,Developed-Low Intensity,Developed-Low Intensity,Developed-Low Intensity,No Dominant Lifeform


In [ ]:
# Aggregate to the DOB from the daily perimeters

In [27]:
# Load the FIRED perimeters (2012-2023)
daily_fp = os.path.join(maindir,'aspen-fire/Aim2/data/spatial/raw/FIRED/fired-daily_west_2012_to_2023.gpkg')
# daily_fp = os.path.join(dest,'fired-daily_west_2012_to_2023.gpkg') # cyverse
daily = gpd.read_file(daily_fp)
daily = daily[daily['id'].isin(aspen_events['id'].unique())] # Work with 2018-> (for Sentinel)
daily = daily.to_crs(proj) # ensure albers projection
daily.columns

Index(['did', 'date', 'id', 'ig_date', 'ig_day', 'ig_month', 'ig_year',
       'last_date', 'event_dur', 'tot_pix', 'tot_ar_km2', 'fsr_px_dy',
       'fsr_km2_dy', 'mx_grw_px', 'mx_grw_km2', 'mx_grw_dte', 'ig_utm_x',
       'ig_utm_y', 'na_l3name', 'geometry'],
      dtype='object')

In [32]:
daily_ = daily[['id','did','date']]
daily_['date'] = pd.to_datetime(daily_['date']).dt.date
daily_.head()

,id,did,date
22,43676,001157351d7551d42ccb77e93b8d7f0d,2021-04-21
54,69951,002b3bc3b445c5fd068ced8fe2f685d8,2018-08-22
58,44848,0031e1e7416cd951ee7f299c490280bc,2018-07-10
71,69813,004769f51032d267b520fec931e64997,2020-02-25
82,69557,00505cd2f06f0c06cafc17d0470c37a5,2020-10-01


In [34]:
daynight_frp = {}
for instrument, df in dob_frp.items():
    df['DATE'] = pd.to_datetime(df['DATE']).dt.date
    frp_event = pd.merge(daily_, df, left_on=['id','date'], right_on=['id','DATE'], how='right')
    daynight_frp[instrument] = frp_event
daynight_frp['SNPP'].head()

,id,did,date,DATE,DAYNIGHT,FRP_MN,FRP_SD,FRP_90,FRP_95,FRP_99
0,492,32040a58c6bb9e5e59019d51a1595a8e,2018-08-10,2018-08-10,N,20.3700,NaN,20.3700,20.3700,20.3700
1,3518,NaN,NaN,2021-08-29,D,13.1654,NaN,13.1654,13.1654,13.1654
2,3518,a19c7fd61b016e4860d5e7ae2b999ba9,2021-08-30,2021-08-30,D,21.1476,NaN,21.1476,21.1476,21.1476
3,3518,a19c7fd61b016e4860d5e7ae2b999ba9,2021-08-30,2021-08-30,N,20.7600,NaN,20.7600,20.7600,20.7600
4,3518,NaN,NaN,2021-09-07,N,1.2900,NaN,1.2900,1.2900,1.2900


In [37]:
len(daynight_frp['MOD61'])

1668

In [ ]:
# # Process each of the archive data products
# gdfs = {} # dictionary to store the geo data frames
# for key, path in dict.items():
#     print(f'Processing: {key}')
#     # Read in the archive vector data
#     vect = glob.glob(path+"*.shp")
#     print([os.path.basename(v) for v in vect])
#     if len(vect) > 1:
#         print("~Merging archive and NRT data.")
#         archive = gpd.read_file([v for v in vect if "archive" in v][0]).to_crs(proj)
#         nrt = gpd.read_file([v for v in vect if "nrt" in v][0]).to_crs(proj)
#         # Concatenate the archive and NRT
#         afd = pd.concat([archive, nrt], ignore_index=True)
#         del archive, nrt
#     else:
#         afd = gpd.read_file(vect[0]).to_crs(proj)

#     # Add some attribute information
#     afd['VID'] = afd.index # unique ID column

#     # Remove low-confidence observations
#     try:
#         afd = afd[afd['CONFIDENCE'] != 'l']
#     except KeyError as e:
#         print(f"KeyError: {e}")

#     del vect

#     #################################################
#     # Extract AFDs within wildfire data (aspen fires)
    
#     # Extract AFDs
#     afd_aspen = gpd.sjoin(afd, fire_buffer, how='inner', predicate='within')
#     print(afd_aspen.columns)
    
#     del afd
    
#     ####################################################
#     # Perform temporal filtering to remove false-positive matches

#     # First, create date columns
#     afd_aspen['ACQ_DATE'] = pd.to_datetime(afd_aspen['ACQ_DATE'])
#     afd_aspen['ACQ_MONTH'] = afd_aspen['ACQ_DATE'].dt.month.astype(int)
#     afd_aspen['ACQ_YEAR'] = afd_aspen['ACQ_DATE'].dt.year.astype(int)
#     afd_aspen['ig_date'] = pd.to_datetime(afd_aspen['ig_date'])
#     afd_aspen['last_date'] = pd.to_datetime(afd_aspen['last_date'])

#     # Filter based on ignition month and year
#     afd_aspen_f = afd_aspen[
#         (afd_aspen['ACQ_YEAR'] >= afd_aspen['ig_date'].dt.year.astype(int)) & 
#         (afd_aspen['ACQ_MONTH'] >= afd_aspen['ig_date'].dt.month.astype(int)) &
#         (afd_aspen['ACQ_YEAR'] <= afd_aspen['last_date'].dt.year.astype(int)) &
#         (afd_aspen['ACQ_MONTH'] <= afd_aspen['last_date'].dt.month.astype(int))
#     ]
    
#     # Keep unique rows
#     afd_aspen_f = afd_aspen_f.drop_duplicates(subset='VID', keep='first')    

#     del afd_aspen
    
#     #############################################
#     # Remove fires with less than 10 observations

#     # Get a count per fire
#     afd_counts = afd_aspen_f.groupby('fired_id').size().reset_index(name='counts')

#     # Get a list of IDs of fires with > 1 obs.
#     ids = afd_counts[afd_counts["counts"] > 1]
    
#     # Grab the new list of unique FIRED ids
#     ids = ids['fired_id'].unique().tolist()
    
#     # Filter the datasets based on these FIRED ids
#     afd_aspen_f = afd_aspen_f[afd_aspen_f['fired_id'].isin(ids)]
#     fired_aspen_f = fired_aspen[fired_aspen['fired_id'].isin(ids)]

#     print(f"Minimum obs./fire: {afd_counts['counts'].min()}; \nMaximum obs./fire: {afd_counts['counts'].max()}")
#     print(f"Number of fires after filtering: {len(fired_aspen_f)}")
#     print(f"Number of obs. after filtering: {len(afd_aspen_f)}")

#     del afd_counts, ids
    
#     #################################################
#     # Plot the distribution of observations over time
#     plt.figure(figsize=(6, 3))
#     afd_aspen_f['ACQ_DATE'].hist(bins=100)
#     plt.title(f'{key} - AFDs (2018-2023)')
#     plt.xlabel('Date')
#     plt.ylabel('Number of Observations')
#     plt.tight_layout()
#     plt.show()
    
#     ##############################
#     # Append to the new dictionary
#     gdfs[key] = afd_aspen_f

# print(f"Total elapsed time: {round((time.time() - t0))} seconds.")

In [ ]:
# Save out the files as they are currently
gdfs['MOD61'].to_file(os.path.join(projdir,'data/spatial/mod/AFD/mod61_archive_afd_aspen.gpkg'))
gdfs['SNPP'].to_file(os.path.join(projdir,'data/spatial/mod/AFD/snpp_archive_afd_aspen.gpkg'))
gdfs['NOAA-20'].to_file(os.path.join(projdir,'data/spatial/mod/AFD/noaa20_archive_afd_aspen.gpkg'))

## Handling acquisition time-of-day and spatially overlapping observations

The VIIRS S-NPP AFD have many overlapping observations during a single fire event. In some cases, the overlapping points are on the same day and time but with different FRP values. In these cases, it may be best to 

From here on we can work with just the SNPP because it has the most consistency across our time period (NOAA-20 started in 2020). Later we can investigate the combination of the three datasets or at least make a comparison.

In [ ]:
# Extract the SNPP observations

snpp_aspen = gdfs['SNPP']

# Filter out FRP == 0
snpp_aspen = snpp_aspen[snpp_aspen['FRP'] > 0]
snpp_aspen.head()


In [ ]:
print(snpp_aspen['FRP'].describe())

In [ ]:
print(snpp_aspen['ACQ_TIME'].describe())

In [ ]:
snpp_aspen = snpp_aspen.reset_index()
snpp_aspen = snpp_aspen.rename(columns={'index':'index_'})
snpp_aspen = snpp_aspen.drop(['index_right'], axis=1)
print(snpp_aspen.columns)

In [ ]:
# Create a datetime object as a new column (in UTC)

import pytz

# Function to convert ACQ_DATE and ACQ_TIME to a datetime object in UTC
def convert_to_datetime(acq_date, acq_time):
    # Ensure ACQ_TIME is in HHMM format
    if len(acq_time) == 3:
        acq_time = '0' + acq_time
    elif len(acq_time) == 2:
        acq_time = '00' + acq_time
    elif len(acq_time) == 1:
        acq_time = '000' + acq_time

    acq_date_str = acq_date.strftime('%Y-%m-%d')
    dt = datetime.strptime(acq_date_str + acq_time, '%Y-%m-%d%H%M')
    dt_utc = pytz.utc.localize(dt)  # Localize the datetime object to UTC
    return dt_utc

# Apply the conversion function to create a new datetime column
snpp_aspen.loc[:, 'ACQ_DATETIME'] = snpp_aspen.apply(lambda row: convert_to_datetime(row['ACQ_DATE'], row['ACQ_TIME']), axis=1)

# Print the resulting GeoDataFrame with timezone-aware datetime objects
print(snpp_aspen['ACQ_DATETIME'].head())

#### Case 1: Same day/time observations with different FRP values (spatially overlapping)

In this case, we have overlapping observations which have the same datetime but (sometimes) different FRP values. To handle this, we can group observations by datetime and perform a dissolve, taking the mean FRP.

In [ ]:
# Perform a spatial intersection to identify overlapping observations
overlap = gpd.sjoin(snpp_aspen, snpp_aspen, predicate='intersects', lsuffix='left', rsuffix='right')
print(overlap.columns)

In [ ]:
# Assign a unique group ID for each set of intersecting observations with the same DATETIME
overlap['_VID_'] = overlap.groupby(['ACQ_DATETIME_left', 'ACQ_DATETIME_right']).ngroup()
print(overlap[['ACQ_DATETIME_left', 'ACQ_DATETIME_right','_VID_']].head())

In [ ]:
# Calculate the 90th percentile of VPD among the observations

# Join the new _VID_ back to the original dataframe using VID
snpp_aspen_ = snpp_aspen.merge(
    overlap[['VID_left', '_VID_']].drop_duplicates(), 
    left_on='VID', right_on='VID_left', how='left').reset_index(drop=True)

# Calculate the 90th percentile FRP for each _VID_
def pct90(group):
    group['FRP_90'] = np.percentile(group['FRP'], 90)
    return group

# Apply the function to calculate the 90th percentile FRP
snpp_aspen_ = snpp_aspen_.groupby('_VID_').apply(pct90).reset_index(drop=True)
snpp_aspen_[['_VID_','VID','ACQ_DATETIME','LATITUDE','LONGITUDE','FRP','FRP_90','VERSION']].head()

In [ ]:
# Dissolve by the same day/time VID to create a new geometry with the 90th percentile FRP

snpp_aspen_dis = snpp_aspen_.dissolve(by='_VID_').reset_index() # this takes the first of each, which should be OK

snpp_aspen_dis.columns


In [ ]:
# Save out a version of these data

# Create the buffered VIIRS obs.
snpp_aspen_plot = snpp_aspen_dis.copy()
snpp_aspen_plot['geometry'] = snpp_aspen_plot.geometry.buffer(375, cap_style=3)  # square buffer 375m

# Save the VIIRS observations (points)
snpp_aspen_dis = snpp_aspen_dis.to_crs(proj)
snpp_aspen_dis.to_file(os.path.join(maindir,'aspen-fire/Aim2/data/spatial/mod/VIIRS/viirs_snpp_pt_fired_events_west_aspen.gpkg'))

# Save the VIIRS observations (plots)
snpp_aspen_plot = snpp_aspen_plot.to_crs(proj)
snpp_aspen_plot.to_file(os.path.join(maindir,'aspen-fire/Aim2/data/spatial/mod/VIIRS/viirs_snpp_plot_fired_events_west_aspen.gpkg'))

## Tidy the FRP data: remove null values, check on the obs./fire, and check on date matches

Some observations may not be joined correctly (i.e., spatial overlap but wrong ignition year, etc). We may also have some fires with too few observations. 

In [ ]:
# Check on the observation counts again
viirs_counts = snpp_aspen_dis.groupby('fired_id').size().reset_index(name='counts')
print(viirs_counts['counts'].min())
print(viirs_counts['counts'].max())

In [ ]:
# Make a map of the fire with the most observations

# Sort the VIIRS counts
viirs_counts = viirs_counts.sort_values('counts', ascending=False).reset_index(drop=True)

# Take the first row (the maximum)
max_obs = viirs_counts.iloc[0]['fired_id']
print(max_obs)

# Filter the fire perimeter and VIIRS obs.
perim = fired_aspen[fired_aspen['fired_id'] == max_obs]
obs = snpp_aspen_dis[snpp_aspen_dis['fired_id'] == max_obs]
obs = obs.copy()
obs['FRP_log'] = np.log1p(obs['FRP'])
obs = obs[obs['DAYNIGHT'] == 'D']
print(len(obs))

# Create the map
fig, ax = plt.subplots(figsize=(4, 5.5))
# Plot VIIRS points
obs.plot(column='FRP_log', ax=ax, legend=True,
         legend_kwds={'label': "Fire Radiative Power (FRP)", 'orientation': "horizontal"},
         cmap='magma', markersize=1, alpha=0.7)
# Plot the fire perimeter
perim.plot(ax=ax, color='none', edgecolor='black', linewidth=1, label='Fire Perimeter')
plt.tight_layout()
plt.grid(True)

# Save the map as a PNG
plt.savefig(os.path.join(maindir,'aspen-fire/Aim2/figures/FigX_MullenFire_FRP.png'), dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
print(len(frp_aspen_f['pct_aspen']))
print(len(frp_aspen_f['FRP']))
      
# Scatterplot of FRP and aspen_pct (fire perimeter)
plt.figure(figsize=(6, 4))  # Set the figure size
plt.scatter(frp_aspen_f['pct_aspen'], frp_aspen_f['FRP'], alpha=0.5)  # Plot with some transparency

# Add titles and labels
plt.ylabel('Fire Radiative Power (FRP)')
plt.xlabel('Aspen %')

plt.show()

## Join VIIRS observations to daily FIRED perimeters

We want to summarize VIIRS observations on a daily basis and then associate them with the correct daily polygon from FIRED. The initial step is to group observations by day.

## Create the VIIRS observation buffer (375m2)

The archive VIIRS data is distributed as shapefiles with centroids representing the pixel center of a VIIRS observation. In order to assess characteristics within the VIIRS observations, we want to create a 375m2 buffer around the centroid locations to approximate the VIIRS pixel size.

In [ ]:
# Create the buffered VIIRS obs.
frp_aspen_plot = frp_aspen_f.copy()
frp_aspen_plot['geometry'] = frp_aspen_plot.geometry.buffer(375, cap_style=3)  # square buffer 375m

print(len(fired_aspen))

# Let's plot one fire using the FRP column to color the "plots"

# Filter the fire perimeter and VIIRS obs.
perim = fired_aspen[fired_aspen['fired_id'] == "42306"]  # Williams Fork Fire "45811.0"
obs = frp_aspen_plot[frp_aspen_plot['fired_id'] == "42306"]
obs = obs.copy()
obs['FRP_log'] = np.log1p(obs['FRP'])
obs = obs[obs['DAYNIGHT'] == 'D']  # plot only daytime observations
print(len(obs))

# Create the map
fig, ax = plt.subplots(figsize=(4, 5.5))
# Plot VIIRS points
obs.plot(column='FRP_log', ax=ax, legend=True,
         legend_kwds={'label': "Fire Radiative Power (FRP)"},
         cmap='magma', markersize=1, alpha=0.7)
# Plot the fire perimeter
perim.plot(ax=ax, color='none', edgecolor='black', linewidth=1, label='Fire Perimeter')
plt.tight_layout()
plt.grid(True)
plt.show()

In [ ]:
centroid = fired_aspen.copy()
centroid['geometry'] = centroid.geometry.centroid

# Make a spatial map of the centroids now
fig, ax = plt.subplots(figsize=(6, 6))

states.plot(ax=ax, edgecolor='black', linewidth=1, color='none')

# Plot centroids
centroid['size'] = centroid['pct_aspen'] * 10  # Adjust the scaling factor as necessary
centroid.plot(
    ax=ax, markersize=centroid['pct_aspen'], 
    column='pct_aspen', cmap='viridis', 
    legend=True, alpha=0.6, 
    legend_kwds={'label': "Aspen Percent"})

# Optional: Plot the original fire perimeters for context
fired_aspen.plot(ax=ax, color='none', edgecolor='gray', linewidth=0.5)

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(True)

del centroid

# Save the map as a PNG
plt.savefig(os.path.join(maindir,'aspen-fire/Aim2/figures/Fig1_aspen_fires.png'), dpi=300, bbox_inches='tight')

plt.show()

## Tidy and save out the necessary files

Now that we have a tidy dataframe for both wildfires with >=5% pre-fire aspen cover and their associated nominal or high confidence VIIRS observations, we can save these files out for further processing. 

Some of the processing will occur in GEE, so for these files we want to save a simplified SHP with only the required attribute information (they will be joined back to the full data after processing).

In [ ]:
# Check on the observation counts again
viirs_counts = frp_aspen_plot.groupby('fired_id').size().reset_index(name='counts')
print(viirs_counts['counts'].min())
print(viirs_counts['counts'].max())

In [ ]:
# Filter the daily files
# Get the list of IDs
ids = fired_aspen['fired_id'].unique()

# Load the daily polygons, subset to aspen fires
daily['id'] = daily['id'].astype(str)
daily = daily[daily['id'].isin(ids)]
print(len(daily['id'].unique()))

# Save the daily wildfire perimeters
daily = daily.to_crs(proj)  # ensure the correct projection before exporting
daily.to_file(os.path.join(maindir,'aspen-fire/Aim2/data/spatial/mod/FIRED/fired_daily_west_aspen.gpkg'))

In [ ]:
# Save the wildfire perimeters
fired_aspen = fired_aspen.to_crs(proj)  # ensure the correct projection before exporting
fired_aspen.to_file(os.path.join(maindir,'aspen-fire/Aim2/data/spatial/mod/FIRED/fired_events_west_aspen.gpkg'))

# Save the VIIRS observations (points)
frp_aspen_f = frp_aspen_f.to_crs(proj)
frp_aspen_f.to_file(os.path.join(maindir,'aspen-fire/Aim2/data/spatial/mod/VIIRS/viirs_obs_fired_events_west_aspen.gpkg'))

# Save the VIIRS observations (plots)
frp_aspen_plot = frp_aspen_plot.to_crs(proj)
frp_aspen_plot.to_file(os.path.join(maindir,'aspen-fire/Aim2/data/spatial/mod/VIIRS/viirs_plots_fired_events_west_aspen.gpkg'))

# Tidy the files for GEE imports

# FIRED perimeters (1km buffer)
print(fired_aspen_1km.columns)
fired_aspen_gee = fired_aspen_1km[['fired_id','ig_date','ig_year','last_date','mx_grw_dte','geometry']]
fired_aspen_gee['ig_date'] = fired_aspen_gee['ig_date'].astype(str)
fired_aspen_gee['last_date'] = fired_aspen_gee['ig_date'].astype(str)
fired_aspen_gee.to_file(os.path.join(maindir,'aspen-fire/Aim2/data/spatial/mod/GEE/fired_events_west_aspen.shp'))

# VIIRS "plots"
print(frp_aspen_plot.columns)
frp_aspen_gee = frp_aspen_plot[['fired_id','VID','ACQ_DATE','DAYNIGHT','geometry']]
frp_aspen_gee['ACQ_DATE'] = frp_aspen_gee['ACQ_DATE'].astype(str)
frp_aspen_gee.to_file(os.path.join(maindir,'aspen-fire/Aim2/data/spatial/mod/GEE/viirs_plots_fired_events_west_aspen.shp'))

print("Success!")

In [ ]:
gc.collect()